# Adiabatic flame temperature

The heat capacity of the gases can be described by the equation
\begin{align*}
\frac{C_p}{R}
&= a_0 + a_1 T + a_2 T^2 + a_3 T^3 + a_4 T^4
\end{align*}
where $T$ is the absolute temperature in kelvin,
$R=8.314\,{\rm J^{-1}\,mol\,K^{-1}}$ is the ideal gas constant, and the
coefficients $a_k$ are given in the data below.

In [12]:
R = 8.314e-3  # ideal gas constant / kJ mol^{-1} K^{-1}
T0 = 298.15   # reference temperature / K
p0 = 1.0e5    # reference pressure / Pa


# Data about the gases: 
#  Mw - molecular weight g/mol
#  Hf - heat of formation kJ/mol
#  Gf - Gibbs free energy kJ/mol
data = {}
data['CO']  = {'Mw':28.01 , 'Hf':-110.5 , 'Gf':-137.2 }
data['CO2'] = {'Mw':44.01 , 'Hf':-393.3 , 'Gf':-394.6 }
data['H2']  = {'Mw': 2.02 , 'Hf':   0.0 , 'Gf':   0.0 }
data['H2O'] = {'Mw':18.02 , 'Hf':-241.8 , 'Gf':-228.4 }
data['CH4'] = {'Mw':16.043, 'Hf': -74.52, 'Gf': -50.45}
data['O2']  = {'Mw':31.999, 'Hf':   0.0 , 'Gf':   0.00}
data['N2']  = {'Mw':28.014, 'Hf':   0.0 , 'Gf':   0.00}


data['CO'] ['Cp_coeff'] = [3.912, -3.913e-3,  1.182e-5, -1.302e-8,  0.515e-11]      
data['CO2']['Cp_coeff'] = [3.259,  1.356e-3,  1.502e-5, -2.374e-8,  1.056e-11]      
data['H2'] ['Cp_coeff'] = [2.883,  3.681e-3, -0.772e-5,  0.692e-8, -0.213e-11]      
data['H2O']['Cp_coeff'] = [4.395, -4.186e-3,  1.405e-5, -1.564e-8,  0.632e-11]
data['CH4']['Cp_coeff'] = [4.568, -8.975e-3,  3.631e-5, -3.407e-8,  1.091e-11]
data['O2'] ['Cp_coeff'] = [3.63 , -1.794e-3,  0.658e-5, -0.601e-8,  0.179e-11]
data['N2'] ['Cp_coeff'] = [3.539, -0.261e-3,  0.007e-5,  0.157e-8, -0.099e-11]

nu = {}
nu['CO']  =  0.0
nu['CO2'] =  1.0
nu['H2']  =  0.0
nu['H2O'] =  2.0
nu['CH4'] = -1.0
nu['O2']  = -2.0
nu['N2']  =  0.0




import numpy as np
import pylab as plt
from scipy.optimize import fsolve


# Understanding the data

Notice that the variable `data` is a dictionary, that has as its keys the names of different molecules and its corresponding values is another dictionary (which contains the properties of the particular molecule).  For example, let's look at the contents of `data` and `data['N2']`

In [ ]:
print(data)
print(data['N2'])

{'CO': {'Mw': 28.01, 'Hf': -110.5, 'Gf': -137.2, 'Cp_coeff': [3.912, -0.003913, 1.182e-05, -1.302e-08, 5.15e-12]}, 'CO2': {'Mw': 44.01, 'Hf': -393.3, 'Gf': -394.6, 'Cp_coeff': [3.259, 0.001356, 1.502e-05, -2.374e-08, 1.056e-11]}, 'H2': {'Mw': 2.02, 'Hf': 0.0, 'Gf': 0.0, 'Cp_coeff': [2.883, 0.003681, -7.72e-06, 6.92e-09, -2.13e-12]}, 'H2O': {'Mw': 18.02, 'Hf': -241.8, 'Gf': -228.4, 'Cp_coeff': [4.395, -0.004186, 1.405e-05, -1.564e-08, 6.32e-12]}, 'CH4': {'Mw': 16.043, 'Hf': -74.52, 'Gf': -50.45, 'Cp_coeff': [4.568, -0.008975, 3.631e-05, -3.407e-08, 1.091e-11]}, 'O2': {'Mw': 31.999, 'Hf': 0.0, 'Gf': 0.0, 'Cp_coeff': [3.63, -0.001794, 6.58e-06, -6.01e-09, 1.79e-12]}, 'N2': {'Mw': 28.014, 'Hf': 0.0, 'Gf': 0.0, 'Cp_coeff': [3.539, -0.000261, 7e-08, 1.57e-09, -9.9e-13]}}
{'Mw': 28.014, 'Hf': 0.0, 'Gf': 0.0, 'Cp_coeff': [3.539, -0.000261, 7e-08, 1.57e-09, -9.9e-13]}


# Calculating the heat capacity

Remember that the expression for the heat capacity is
\begin{align*}
\frac{C_p}{R}
&= a_0 + a_1 T + a_2 T^2 + a_3 T^3 + a_4 T^4
\end{align*}
If we put the coefficients $a_n$ in a list called `a`, then
we can write Python code to calculate the heat capacity that looks extremely similar to what is written mathematically.  This is what we do in the code snippet below, using the coefficients for nitrogen ('N2'):


In [13]:
a = data['N2'] ['Cp_coeff']

print(a)
print(f'a_0 = a[0]')

T = 300
Cp = a[0] + a[1]*T + a[2]*T**2 + a[3]*T**3 + a[4]*T**4
Cp *= R 

[3.539, -0.000261, 7e-08, 1.57e-09, -9.9e-13]
a_0 = a[0]


## Creating a function

So, given the list of coefficients for the heat capacity, we can use Python to calculate $C_p$.  Now, we do not just want to do this once, so we can indent everything we did and move it into a function, which we will call `get_Cp_pure`.  This function will take two arguments.  The first argument will be the (absolute) temperature at which we will calculate the heat capacity, and the second argument will be the name of the molecule that we want.

In [26]:
def get_Cp_pure(T, molecule):
  a = data[molecule] ['Cp_coeff']
  Cp = a[0] + a[1]*T + a[2]*T**2 + a[3]*T**3 + a[4]*T**4
  Cp *= R
  return Cp


print(f"C_p of nitrogen = {get_Cp_pure(300, 'N2')} kJ mol^{-1} K^{-1}")   

print()
T = 300.0
for mol in data.keys():
  print(f"{mol}: C_p = {get_Cp_pure(T, mol)} kJ mol^{-1} K^{-1}") 


C_p of nitrogen = 0.029110398494000003 kJ mol^-1 K^-1

CO: C_p = 0.029033111550000004 kJ mol^-1 K^-1
CO2: C_p = 0.037098364984 kJ mol^-1 K^-1
H2: C_p = 0.028783791317999997 kJ mol^-1 K^-1
H2O: C_p = 0.033527136168 kJ mol^-1 K^-1
CH4: C_p = 0.035848978634 kJ mol^-1 K^-1
O2: C_p = 0.029400207906000002 kJ mol^-1 K^-1
N2: C_p = 0.029110398494000003 kJ mol^-1 K^-1


Before we move on, let's try to write the expression for the heat capacity in a more compact form.  If we remember our summation notation, we can write
\begin{align*}
\frac{C_p}{R}
&= \sum_{n=0}^4 a_n T^n
\end{align*}
We can mimic this form by using a loop to perform the summation:

In [25]:
def get_Cp_pure(T, molecule):
  a = data[molecule] ['Cp_coeff']
  Cp = 0.0
  for n in range(len(a)):
      Cp += a[n]*T**n
  Cp *= R
  return Cp


T = 300.0
for mol in data.keys():
  print(f"{mol}: C_p = {get_Cp_pure(T, mol)} kJ mol^{-1} K^{-1}")   

CO: C_p = 0.029033111550000004 kJ mol^-1 K^-1
CO2: C_p = 0.037098364984 kJ mol^-1 K^-1
H2: C_p = 0.028783791317999997 kJ mol^-1 K^-1
H2O: C_p = 0.033527136168 kJ mol^-1 K^-1
CH4: C_p = 0.035848978634 kJ mol^-1 K^-1
O2: C_p = 0.029400207906000002 kJ mol^-1 K^-1
N2: C_p = 0.029110398494000003 kJ mol^-1 K^-1


# Calculating the enthalpy

There was really no reason to use a loop.  It just makes things look nicer.  However, often when we make things look nicer, it makes things easier to see and do.  As an example, let's see how we can calculate the enthalpy.

The enthalpy is just the integral of the heat capacity:
\begin{align*}
H(T) &= H(T_0) + \int_{T_0}^T dT'\, C_p(T')
\\
&=
H(T_0) + \int_{T_0}^T dT'\, R\sum_{n=0}^4 a_n T'^n
\\
&=
H(T_0) + R\sum_{n=0}^4 a_n \frac{T'^{n+1}}{n+1}
\Bigg]_{T_0}^T
\\
&=
H(T_0) 
+ R\sum_{n=0}^4 \frac{a_n}{n+1} (T^{n+1}-T_0^{n+1})
\end{align*}
Note that in this problem $T_0=298.15$, and the enthalpy at this temperature is the enthalpy of formation (i.e. $H_f=H(T_0)$).

We can take this relation to create a function that gives us the enthalpy of a species:


In [22]:
def get_H_pure(T, molecule):
  a = data[molecule]['Cp_coeff']
  H = data[molecule]['Hf']
  for n in range(len(a)):
    H += R*a[n]*(T**(n+1)-T0**(n+1))/(n+1)
  return H

T = 298.15
for mol in data.keys():
  print(f"{mol}: H = {get_H_pure(T, mol)} kJ/mol")

CO: H = -110.5 kJ/mol
CO2: H = -393.3 kJ/mol
H2: H = 0.0 kJ/mol
H2O: H = -241.8 kJ/mol
CH4: H = -74.52 kJ/mol
O2: H = 0.0 kJ/mol
N2: H = 0.0 kJ/mol


# Mixtures

We are now able to compute the enthalpy and heat capacity of any pure species in the `data` dictionary at any temperature.  We will now try to calculate the enthalpy and heat capacity of a mixture.

First, let's define a function that will return the dot product between two vectors:

In [23]:
def dot_product(vec1, vec2):

  prod_vec = [x1*x2 for x1, x2 in zip(vec1, vec2)]
  return sum(prod_vec)

v1 = [1, 2, 3]
v2 = [1, 1, 1]
w = [3, 4, 5]
dot = dot_product(w, v2)
print(dot)

12


Using this dot product, we can then determine the heat capacity of a mixture by creating a "vector" `N_list`, with each "component" consisting of the mole numbers of each species, and another "vector" `Cp_list`, with each "component" consisting of the molar heat capacity of each pure species.  The dot product between these two vectors is then the heat capacity of the mixture.  We can do something similar to determine the enthalpy of the mixture.

In [24]:
N_dict = {molecule: 0.0 for molecule in data.keys()}
N_dict['CH4'] = 1.0
N_dict['O2'] = 0.21*10
N_dict['N2'] = 0.79*10

T = 300
N_list = [N_dict[molecule] for molecule in data.keys()]             # create a "vector" of mole numbers
Cp_list = [get_Cp_pure(T, molecule) for molecule in data.keys()]    # create a "vector" of specific heat capacities
H_list = [get_H_pure(T, molecule) for molecule in data.keys()]      # create a "vector" of specific enthalpies

#print(H)
Cp = dot_product(N_list, Cp_list)
H = dot_product(N_list, H_list)

print(Cp, H)

0.32756156333920006 -73.91411151112823


Now we create functions from these calculations:

In [34]:
def get_H(T, N_dict):

  N_list = [N_dict[molecule] for molecule in data.keys()]
  H_list = [get_H_pure(T, molecule) for molecule in data.keys()] 

  H = dot_product(N_list, H_list)
  return H


N_in = {molecule: 0.0 for molecule in data.keys()}
N_in['O2'] = 0.21
N_in['N2'] = 0.79
print(get_H(300, N_in))






0.0539635829751982


# Energy balance

We can perform energy balances with these functions.  Let's consider a system where we have an inlet stream of 1.0 mol/s of methane and 10 mol/s of air at a temperature $T_{\rm in}=300\,{\rm K}$, and an outlet stream of 1.0 mol/s of methane and 10 mol/s of air.  If we add heat to the system at a rate $\dot{Q}=10\,{\rm kW}$, what is the temperature of the outlet stream?

In [39]:

def energy_balance(T):
    N_in = {molecule: 0.0 for molecule in data.keys()}
    N_in['CH4'] = 1.0       # inlet species flowrate in mol/s
    N_in['O2'] = 0.21*10
    N_in['N2'] = 0.79*10
    Tin = 300   # inlet temperature / K
    Q = 10      # rate of heat added to the system in kW
    Hin = get_H(Tin, N_in)
    Hout = get_H(T, N_in)   # note that in this case the outlet mole number is the same as the inlet mole numbers
    return Hout - Hin - Q

Tguess = 300
solution = fsolve(energy_balance, Tguess)
Tout = solution[0]
print(f'outlet temperature = {Tout} K')



outlet temperature = 330.43981516578293 K


# Adding a chemical reaction

Now let's look at a chemical reaction: the combustion of methane.
\begin{align*}
{\rm CH}_4 + 2{\rm O}_2
\longrightarrow
2{\rm H_2O} + {\rm CO_2}
\end{align*}

If we have a reaction in the system, then the outlet mole numbers will be related to the inlet mole numbers through the relations
\begin{align*}
N_{\rm CH_4}^{\rm out}
&= 
N_{\rm CH_4}^{\rm in} - \alpha
\\
N_{\rm O_2}^{\rm out}
&= 
N_{\rm O_2}^{\rm in} -2 \alpha
\\
N_{\rm H_2O}^{\rm out}
&= 
N_{\rm H_2O}^{\rm in} + 2\alpha
\\
N_{\rm CO_2}^{\rm out}
&= 
N_{\rm CO_2}^{\rm in} + \alpha
\end{align*}
where $\alpha$ is the extent of reaction.


This can be written in terms of the stoichiometric coefficients of the combustion reaction as:
\begin{align*}
N_{\rm CH_4}^{\rm out}
&= 
N_{\rm CH_4}^{\rm in} +  \nu_{\rm CH_4} \alpha
\\
N_{\rm O_2}^{\rm out}
&= 
N_{\rm O_2}^{\rm in} +  \nu_{\rm O_2} \alpha
\\
N_{\rm H_2O}^{\rm out}
&= 
N_{\rm H_2O}^{\rm in} + \nu_{\rm H_2O} \alpha
\\
N_{\rm CO_2}^{\rm out}
&= 
N_{\rm CO_2}^{\rm in} + \nu_{\rm CO_2} \alpha
\end{align*}
where the stoichiometric coefficient is positive for products and
negative for reactants.

More generally, we can write this as:
\begin{align*}
N_{\rm mol}^{\rm out}
&= 
N_{\rm mol}^{\rm in} + \alpha \nu_{\rm mol}
\end{align*}
where mol is any molecule in the system.  Note that the stoichiometric coefficient for any species that does not participate in the reaction (e.g., nitrogen) is equal to zero.

The stoichiometric constants for the reaction are given in the variable `nu`, which was defined at the top of this notebook.

We can include the presence of a chemical reaction in our energy balance as:

In [40]:

def energy_balance(T):
    Nin = {molecule: 0.0 for molecule in data.keys()}
    Nin['CH4'] = 1.0        # these are the inlet species flow rates in mol/s 
    Nin['O2'] = 0.21 * 10
    Nin['N2'] = 0.79 * 10
    Tin = 300   # inlet temperature / K
    Q = 10
    Hin = get_H(Tin, Nin)
    alpha = 0.2   # the extent of reaction is only 2 mol/s
    Nout = {mol: Nin[mol] + alpha*nu[mol] for mol in data.keys()}
    Hout = get_H(T, Nout)
    return Hout - Hin - Q

Tguess = 300
solution = fsolve(energy_balance, Tguess)
Tout = solution[0]
print(f'outlet temperature = {Tout} K')


outlet temperature = 783.5740003592584 K


# Conclusions

In this notebook, we gradually built up functions for calculating the molar heat capacity and molar enthalpy of any ideal gas mixture at any temperature.  Then we used these functions to perform energy balance calculations.  

There are many, many different ways in which this basic code can be extended.  For example, can you include multiple input and output streams?  Can you write a code for a general number of in/out streams?  Can you include additional chemical reactions (e.g., the partial combustion of methane to form carbon monoxide or the combustion of hydrogen)?